In [ ]:
import git
import pandas as pd

pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv").to_csv("time_series_covid19_confirmed_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv").to_csv("time_series_covid19_confirmed_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv").to_csv("time_series_covid19_deaths_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv").to_csv("time_series_covid19_deaths_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv").to_csv("time_series_covid19_recovered_global.csv", index=False)






data_confirmed = pd.read_csv("time_series_covid19_confirmed_global.csv")
data_recovered = pd.read_csv("time_series_covid19_recovered_global.csv")
data_death     = pd.read_csv("time_series_covid19_deaths_global.csv")


#data_salida_confirmed = data_confirmed[["Province/State","Country/Region","Lat","Long"]]
#data_salida_recovered = data_recovered[["Province/State","Country/Region","Lat","Long"]]
#data_salida_death = data_death[["Province/State","Country/Region","Lat","Long"]]

data_salida_confirmed = pd.DataFrame() 
data_salida_recovered = pd.DataFrame()
data_salida_death = pd.DataFrame()
data_salida_confirmed["Province/State"] = data_confirmed["Province/State"] 
data_salida_confirmed["Country/Region"] = data_confirmed["Country/Region"] 
data_salida_confirmed["Lat"] = data_confirmed["Lat"]
data_salida_confirmed["Long"] = data_confirmed["Long"]

#data_salida_recovered["Province/State"] = data_recovered["Province/State"] 
data_salida_recovered["Country/Region"] = data_recovered["Country/Region"] 
#data_salida_recovered["Lat"] = data_recovered["Lat"]
#data_salida_recovered["Long"] = data_recovered["Long"]

#data_salida_death["Province/State"] = data_death["Province/State"] 
data_salida_death["Country/Region"] = data_death["Country/Region"] 
#data_salida_death["Lat"] = data_death["Lat"]
#data_salida_death["Long"] = data_death["Long"]


columna_anterior = ""
for columna in data_confirmed.columns[5:]:
    if(columna_anterior == ""):
        #print("no se hace nada")
        pass
    else:
        #print(columna + " y " + columna_anterior)
        data_salida_confirmed[columna] = data_confirmed[columna] - data_confirmed[columna_anterior]
        data_salida_recovered[columna] = data_recovered[columna] - data_recovered[columna_anterior]
        data_salida_death[columna]     = data_death[columna]     - data_death[columna_anterior]
    columna_anterior = columna
    
entrada = {"Province/State":""}   #,"Country/Region":"","Lat":"","Long":""}
salida = []
#index = 0
#data_salida = pd.DataFrame()

data_salida = pd.DataFrame(columns=data_salida_confirmed.columns[:4])

lista_confirmed = list(data_salida_confirmed.iterrows())
lista_recovered = list(data_salida_recovered.iterrows())
lista_death = list(data_salida_death.iterrows())

n = 0
for i in range(len(lista_confirmed)):
    for columna in data_salida_confirmed.columns[4:]:         
        #print(columna)
        #print(i[1][columna])
        entrada["Province/State"] = lista_confirmed[i][1]["Province/State"]
        entrada["Country/Region"] = lista_confirmed[i][1]["Country/Region"]
        #"{0:.2f}".format(division_con_decimales)
        if(round(lista_confirmed[i][1]["Lat"]) == lista_confirmed[i][1]["Lat"]):
            entrada["Lat"] = round(lista_confirmed[i][1]["Lat"])
        else:
            entrada["Lat"] = lista_confirmed[i][1]["Lat"]
        if(round(lista_confirmed[i][1]["Long"]) == lista_confirmed[i][1]["Long"]):
            entrada["Long"] = round(lista_confirmed[i][1]["Long"])
        else:
            entrada["Long"] = lista_confirmed[i][1]["Long"]
        #entrada["Lat"] = "{0:.6f}".format(lista_confirmed[i][1]["Lat"])
        #entrada["Long"] = "{0:.6f}".format(lista_confirmed[i][1]["Long"])
        entrada["fecha"] = columna
        entrada["confirmados"] = lista_confirmed[i][1][columna]
        entrada["fallecidos"] = lista_death[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        
        #entrada["recuperados"] = lista_recovered[i][1][columna]
        salida.append(data_salida.append(entrada,ignore_index=True))
data_salida = pd.concat(salida)

entrada = {"Province/State":""}
salida = []
data_salida_aux = pd.DataFrame(columns=data_salida_confirmed.columns[:4])
for i in range(len(lista_recovered)):
    for columna in data_salida_recovered.columns[4:]:
        entrada["Province/State"] = lista_confirmed[i][1]["Province/State"]
        entrada["Country/Region"] = lista_confirmed[i][1]["Country/Region"]
        entrada["fecha"] = columna
        entrada["recuperado"] = lista_recovered[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        salida.append(data_salida_aux.append(entrada, ignore_index=True))
data_salida_aux = pd.concat(salida)
del data_salida_aux["Province/State"]
del data_salida_aux["Country/Region"]
del data_salida_aux["fecha"]
del data_salida_aux["Lat"]
del data_salida_aux["Long"]


merged_left = pd.merge(left=data_salida, right=data_salida_aux, how='left', left_on='codigo', right_on='codigo')
merged_left

merged_left.to_csv("acumuladoLuis.csv", index=False)


repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
print(repoLocal.git.status())
repoLocal.git.add(".")
try:
    repoLocal.git.commit(m='Update automatico via python')
except:
    pass

origin = repoLocal.remote(name='origin')
origin.push()